# Appendix 10.2: Tool Use

- [Lesson](#lesson)
- [Exercises](#exercises)
- [Example Playground](#example-playground)

## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [ ]:
import os
import re
from openai import OpenAI

# Retrieve the MODEL_NAME variable from the IPython store
%store -r MODEL_NAME

# Client for Qwen (DashScope-compatible endpoint)
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"), base_url="https://dashscope.aliyuncs.com/compatible-mode/v1")

def get_completion(messages, system_prompt="", prefill=""):
    if system_prompt:
        messages_with_system = [{"role": "system", "content": system_prompt}] + messages
    else:
        messages_with_system = messages
    
    if prefill:
        messages_with_system.append({"role": "assistant", "content": prefill})

    response = client.chat.completions.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=messages_with_system,
    )
    print(response.model_dump_json())
    return response.choices[0].message.content

---

## Lesson

While it might seem conceptually complex at first, tool use, a.k.a. function calling, is actually quite simple! You already know all the skills necessary to implement tool use, which is really just a combination of substitution and prompt chaining.

In previous substitution exercises, we substituted text into prompts. With tool use, we substitute tool or function results into prompts. Qwen can't literally call or access tools and functions. Instead, we have Qwen:
1. Output the tool name and arguments it wants to call
2. Halt any further response generation while the tool is called
3. Then we reprompt with the appended tool results

Function calling is useful because it expands Qwen's capabilities and enables Qwen to handle much more complex, multi-step tasks.
Some examples of functions you can give Qwen:
- Calculator
- Word counter
- SQL database querying and data retrieval
- Weather API

You can get Qwen to do tool use by combining these two elements:

1. A system prompt, in which we give Qwen an explanation of the concept of tool use as well as a detailed descriptive list of the tools it has access to
2. The control logic with which to orchestrate and execute Qwen's tool use requests

### Tool use roadmap

*This lesson teaches our current tool use format. However, we will be updating and improving tool use functionality in the near future, including:*
* *A more streamlined format for function definitions and calls*
* *More robust error handilgj and edge case coverage*
* *Tighter integration with the rest of our API*
* *Better reliability and performance, especially for more complex tool use tasks*

### Examples

To enable tool use in Qwen, we start with the system prompt. In this special tool use system prompt, we tell Qwen:
* The basic premise of tool use and what it entails
* How Qwen can call and use the tools it's been given
* A detailed list of tools it has access to in this specific scenario 

Here's the first part of the system prompt, explaining tool use to Qwen. This part of the system prompt is generalizable across all instances of prompting Qwen for tool use. The tool calling structure we're giving Qwen (`<tool_call> [...] </tool_call>`) is a structure that Qwen can be trained to use, so we recommend that you stick with this.

In [ ]:
system_prompt_tools_general_explanation = """You are an intelligent assistant that can call various tools to help users solve problems. You can select appropriate tools based on user requirements and call them correctly.

When you need to call a function, return a JSON object with function name and arguments within <tool_call></tool_call> XML tags:
<tool_call>
{"name": <function-name>, "arguments": <args-json-object>}
</tool_call>

For each function call:
1. Provide the function name that matches the available tools
2. Provide arguments as a JSON object with the required parameters
3. Use proper JSON formatting for all argument values

The function results will be returned to you, and you should incorporate them into your response to answer the user's question."""

Here's the second part of the system prompt, which defines the exact tools Qwen has access to in this specific situation. In this example, we will be giving Qwen a calculator tool, which takes three parameters: two operands and an operator. 

Then we combine the two parts of the system prompt.

In [ ]:
system_prompt_tools_specific_tools = """Here are the functions signatures within <tools></tools> XML tags:

<tool_call>
{
    "type": "function",
    "function": {
      "name": "calculator",
      "description": "Calculator function for doing basic arithmetic. Supports addition, subtraction, multiplication, and division.",
      "parameters": {
        "type": "object",
        "properties": {
          "first_operand": {
            "type": "number",
            "description": "First operand (before the operator)"
          },
          "second_operand": {
            "type": "number",
            "description": "Second operand (after the operator)"
          },
          "operator": {
            "type": "string",
            "description": "The operation to perform. Must be either +, -, *, or /"
          }
        },
        "required": ["first_operand", "second_operand", "operator"]
      }
    }
  }
</tool_call>
"""

system_prompt = system_prompt_tools_general_explanation + system_prompt_tools_specific_tools

Now we can give Qwen a question that requires use of the `calculator` tool. We will use `<function_calls\>` in `stop_sequences` to detect if and when Qwen calls the function.

In [ ]:
multiplication_message = {
    "role": "user",
    "content": "Multiply 1,984,135 by 9,343,116"
}

# Get Qwen's response
function_calling_response = get_completion([multiplication_message], system_prompt=system_prompt)
print(function_calling_response)

Now, we can extract out the parameters from Qwen's function call and actually run the function on Qwen's behalf.

First we'll define the function's code.

In [ ]:
def do_pairwise_arithmetic(num1, num2, operation):
    if operation == '+':
        return num1 + num2
    elif operation == "-":
        return num1 - num2
    elif operation == "*":
        return num1 * num2
    elif operation == "/":
        return num1 / num2
    else:
        return "Error: Operation not supported."

Then we'll extract the parameters from Qwen's function call response. If all the parameters exist, we run the calculator tool.

In [ ]:
import json

def extract_tool_call(message):
    """Extract tool call from Qwen's response in JSON format"""
    # Find the tool_call tags
    start = message.find("<tool_call>")
    end = message.find("</tool_call>")
    
    if start == -1 or end == -1:
        return None
    
    # Extract JSON between tags
    json_str = message[start + len("<tool_call>"):end].strip()
    
    try:
        return json.loads(json_str)
    except json.JSONDecodeError:
        return None

# Extract tool call from response
tool_call = extract_tool_call(function_calling_response)

if tool_call:
    function_name = tool_call.get("name")
    arguments = tool_call.get("arguments", {})
    
    first_operand = arguments.get("first_operand")
    second_operand = arguments.get("second_operand")
    operator = arguments.get("operator")
    
    if first_operand is not None and second_operand is not None and operator:
        result = do_pairwise_arithmetic(first_operand, second_operand, operator)
        print("---------------- RESULT ----------------")
        print(f"{result:,}")
    else:
        print("Missing parameters in tool call")
else:
    print("No tool call found in response")

Now that we have a result, we have to properly format that result so that when we pass it back to Qwen, Qwen understands what tool that result is in relation to. In Qwen's JSON format, we structure the result as JSON:
```json
{
  "tool_name": "{TOOL_NAME}",
  "result": {TOOL_RESULT}
}
```

Run the cell below to format the above tool result into this structure.

In [ ]:
def construct_tool_result(tool_name, tool_result):
    """Format tool result in JSON format"""
    return json.dumps({
        "tool_name": tool_name,
        "result": tool_result
    }, ensure_ascii=False, indent=2)

if tool_call:
    function_results = construct_tool_result("calculator", result)
    print(function_results)

Now all we have to do is send this result back to Qwen by appending the result to the same message chain as before, and we're good!

In [ ]:
full_first_response = function_calling_response

# Construct the full conversation
messages = [
    multiplication_message,
    {"role": "assistant", "content": full_first_response},
    {"role": "user", "content": function_results}
]
   
# Print Qwen's response
final_response = get_completion(messages, system_prompt=system_prompt)
print("------------- FINAL RESULT -------------")
print(final_response)

Congratulations on running an entire tool use chain end to end!

Now what if we give Qwen a question that doesn't that doesn't require using the given tool at all?

In [ ]:
non_multiplication_message = {
    "role": "user",
    "content": "Tell me the capital of France."
}

# Get Qwen's response
function_calling_response = get_completion([non_multiplication_message], system_prompt=system_prompt)
print(function_calling_response)

Success! As you can see, Qwen knew not to call the function when it wasn't needed.

If you would like to experiment with the lesson prompts without changing any content above, scroll all the way to the bottom of the lesson notebook to visit the [**Example Playground**](#example-playground).

---

## Exercises
- [Exercise 10.2.1 - SQL](#exercise-1021---SQL)

### Exercise 10.2.1 - SQL
In this exercise, you'll be writing a tool use prompt for querying and writing to the world's smallest "database". Here's the initialized database, which is really just a dictionary.

In [ ]:
db = {
    "users": [
        {"id": 1, "name": "Alice", "email": "alice@example.com"},
        {"id": 2, "name": "Bob", "email": "bob@example.com"},
        {"id": 3, "name": "Charlie", "email": "charlie@example.com"}
    ],
    "products": [
        {"id": 1, "name": "Widget", "price": 9.99},
        {"id": 2, "name": "Gadget", "price": 14.99},
        {"id": 3, "name": "Doohickey", "price": 19.99}
    ]
}

And here is the code for the functions that write to and from the database.

In [ ]:
def get_user(user_id):
    for user in db["users"]:
        if user["id"] == user_id:
            return user
    return None

def get_product(product_id):
    for product in db["products"]:
        if product["id"] == product_id:
            return product
    return None

def add_user(name, email):
    user_id = len(db["users"]) + 1
    user = {"id": user_id, "name": name, "email": email}
    db["users"].append(user)
    return user

def add_product(name, price):
    product_id = len(db["products"]) + 1
    product = {"id": product_id, "name": name, "price": price}
    db["products"].append(product)
    return product

To solve the exercise, start by defining a system prompt like `system_prompt_tools_specific_tools` above. Make sure to include the name and description of each tool, along with the name and type and description of each parameter for each function. We've given you some starting scaffolding below.

In [ ]:
system_prompt_tools_specific_tools_sql = """
"""

system_prompt = system_prompt_tools_general_explanation + system_prompt_tools_specific_tools_sql

When you're ready, you can try out your tool definition system prompt on the examples below. Just run the below cell!

In [ ]:
examples = [
    "Add a user to the database named Deborah.",
    "Add a product to the database named Thingo",
    "Tell me the name of User 2",
    "Tell me the name of Product 3"
]

for example in examples:
    message = {
        "role": "user",
        "content": example
    }

    # Get & print Qwen's response
    function_calling_response = get_completion([message], system_prompt=system_prompt)
    print(example, "\n----------\n\n", function_calling_response, "\n*********\n*********\n*********\n\n")

If you did it right, the function calling messages should call the `add_user`, `add_product`, `get_user`, and `get_product` functions correctly.

For extra credit, add some code cells and write parameter-parsing code. Then call the functions with the parameters Qwen gives you to see the state of the "database" after the call.

❓ If you want to see a possible solution, run the cell below!

In [ ]:
from hints import exercise_10_2_1_solution; print(exercise_10_2_1_solution)

### Congrats!

Congratulations on learning tool use and function calling! Head over to the last appendix section if you would like to learn more about search & RAG.

---

## Example Playground

This is an area for you to experiment freely with the prompt examples shown in this lesson and tweak prompts to see how it may affect Claude's responses.

In [ ]:
system_prompt_tools_general_explanation = """You are an intelligent assistant that can call various tools to help users solve problems. You can select appropriate tools based on user requirements and call them correctly.

When you need to call a function, return a JSON object with function name and arguments within <tool_call></tool_call> XML tags:
<tool_call>
{"name": <function-name>, "arguments": <args-json-object>}
</tool_call>

For each function call:
1. Provide the function name that matches the available tools
2. Provide arguments as a JSON object with the required parameters
3. Use proper JSON formatting for all argument values

The function results will be returned to you, and you should incorporate them into your response to answer the user's question."""

In [ ]:
system_prompt_tools_specific_tools = """Here are the functions available in JSON format:

[
  {
    "type": "function",
    "function": {
      "name": "calculator",
      "description": "Calculator function for doing basic arithmetic. Supports addition, subtraction, multiplication, and division.",
      "parameters": {
        "type": "object",
        "properties": {
          "first_operand": {
            "type": "number",
            "description": "First operand (before the operator)"
          },
          "second_operand": {
            "type": "number",
            "description": "Second operand (after the operator)"
          },
          "operator": {
            "type": "string",
            "description": "The operation to perform. Must be either +, -, *, or /"
          }
        },
        "required": ["first_operand", "second_operand", "operator"]
      }
    }
  }
]
"""

system_prompt = system_prompt_tools_general_explanation + system_prompt_tools_specific_tools

In [ ]:
multiplication_message = {
    "role": "user",
    "content": "Multiply 1,984,135 by 9,343,116"
}

# Get Qwen's response
function_calling_response = get_completion([multiplication_message], system_prompt=system_prompt)
print(function_calling_response)

In [ ]:
def do_pairwise_arithmetic(num1, num2, operation):
    if operation == '+':
        return num1 + num2
    elif operation == "-":
        return num1 - num2
    elif operation == "*":
        return num1 * num2
    elif operation == "/":
        return num1 / num2
    else:
        return "Error: Operation not supported."

In [ ]:
def find_parameter(message, parameter_name):
    parameter_start_string = f"name=\"{parameter_name}\">"
    start = message.index(parameter_start_string)
    if start == -1:
        return None
    if start > 0:
        start = start + len(parameter_start_string)
        end = start
        while message[end] != "<":
            end += 1
    return message[start:end]

first_operand = find_parameter(function_calling_response, "first_operand")
second_operand = find_parameter(function_calling_response, "second_operand")
operator = find_parameter(function_calling_response, "operator")

if first_operand and second_operand and operator:
    result = do_pairwise_arithmetic(int(first_operand), int(second_operand), operator)
    print("---------------- RESULT ----------------")
    print(f"{result:,}")

In [ ]:
def construct_successful_function_run_injection_prompt(invoke_results):
    constructed_prompt = (
        "<function_results>\n"
        + '\n'.join(
            f"<result>\n<tool_name>{res['tool_name']}</tool_name>\n<stdout>\n{res['tool_result']}\n</stdout>\n</result>"
            for res in invoke_results
        ) + "\n</function_results>"
    )

    return constructed_prompt

formatted_results = [{
    'tool_name': 'do_pairwise_arithmetic',
    'tool_result': result
}]
function_results = construct_successful_function_run_injection_prompt(formatted_results)
print(function_results)

In [ ]:
full_first_response = function_calling_response + "</function_calls>"

# Construct the full conversation
messages = [multiplication_message,
{
    "role": "assistant",
    "content": full_first_response
},
{
    "role": "user",
    "content": function_results
}]
   
# Print Qwen's response
final_response = get_completion(messages, system_prompt=system_prompt)
print("------------- FINAL RESULT -------------")
print(final_response)

In [ ]:
non_multiplication_message = {
    "role": "user",
    "content": "Tell me the capital of France."
}

# Get Qwen's response
function_calling_response = get_completion([non_multiplication_message], system_prompt=system_prompt)
print(function_calling_response)